In [32]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

torch.set_default_device('cuda')

In [33]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

# Load Data

In [34]:
train_data = pd.read_csv('train.csv')
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
X = train_data.drop('label', axis=1).values
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [36]:
y = train_data['label'].values
y_tensor = torch.tensor(y, dtype=torch.long)
y_tensor

tensor([1, 0, 1,  ..., 7, 6, 9], device='cuda:0')

In [43]:
test_data = pd.read_csv('test.csv')
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
X_test = torch.tensor(test_data.values, dtype=torch.float32)
X_test

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [44]:
train_dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=torch.Generator(device='cuda'))

In [47]:
test_dataset = TensorDataset(X_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Write Model

In [38]:
class DigitClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(DigitClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
input_size = X.shape[1]
hidden_size = 128
num_classes = 10

In [40]:
model = DigitClassifier(input_size, hidden_size, num_classes)
model

DigitClassifier(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [41]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.1026
Epoch [2/10], Loss: 0.1874
Epoch [3/10], Loss: 0.5056
Epoch [4/10], Loss: 0.0198
Epoch [5/10], Loss: 0.0429
Epoch [6/10], Loss: 0.0520
Epoch [7/10], Loss: 0.0391
Epoch [8/10], Loss: 0.0087
Epoch [9/10], Loss: 0.0015
Epoch [10/10], Loss: 0.0001


In [42]:
model.eval()

DigitClassifier(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [55]:
predictions = []

with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch[0]
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())

predictions_df = pd.DataFrame(predictions, columns=['Label'])
predictions_df.index += 1
predictions_df.index.name = 'ImageId'
predictions_df

,Label
1,1
2,0
3,1
4,4
5,0
...,...
41996,0
41997,1
41998,7
41999,6


In [54]:
predictions_df.to_csv('submission.csv', index=True)